## Analyze A/B Test Exercise


## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  

In this project I will go through the results of an A/B test run by an e-commerce website.  The goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.  
  

<a id='probability'></a>
#### Part I - Probability

First I will import the libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Reading in the `ab_data.csv` data. This will be stored in `df`.   
The first few lines of the dataset will be shown.

In [ ]:
#The dataset is read and the first few rows are shown
df = pd.read_csv('ab_data.csv')
df.head()

The below cell is used to find the number of rows in the dataset.

In [ ]:
#The number of rows in the dataset if found
df.shape[0]

The number of unique users in the dataset.

In [ ]:
#The amount of unique users are found
df['user_id'].nunique()

The proportion of users converted.

In [ ]:
#The amount of converted users is found
df.query('converted == 1').count()[0]/df.count()[0]

The number of times the `new_page` and `treatment` don't line up.

In [ ]:
#A query is used to investigate when new_page and treatment does not line up
df.query("group == 'treatment' and landing_page == 'old_page'").count()[0] + df.query("group == 'control' and landing_page == 'new_page'").count()[0]


Do any of the rows have missing values?

In [ ]:
#Investigating if any of the rows have missing values
df.isnull().sum()

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, it is not certain if this row truly received the new or old page.  

a. Removing values that are not adhering to expected values and store the new dataframe in **df2**.

In [ ]:
#Removing data that is not adhering to the expected values
df2 = df.query("group == 'treatment' and landing_page == 'new_page' or group == 'control' and landing_page == 'old_page'")
df2.head()

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Using **df2** and the cells below to answer the posed questions

a. How many unique **user_id**s are in **df2**?

In [ ]:
#The amount of unique user id's is investigated
df2['user_id'].nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
#The single duplicate is shown
df2[df2.duplicated(['user_id'], keep=False)]

c. What is the row information for the repeat **user_id**? 

In [ ]:
#See above

d. Remove **one** of the rows with a duplicate **user_id**, but keep the dataframe as **df2**.

In [ ]:
#Removing one of the identified duplicate rows
df2 = df2.drop(1899)
df2.info()

`4.` Use **df2** in the below cells to answer the posed questions.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
#Conversion rate
df2.query('converted == 1').count()[0]/df2.count()[0]

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
#Conversion rate for the control group, where the mean can be used as the conversion rate
df2.groupby('group').describe()

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
#See mean value above

d. What is the probability that an individual received the new page?

In [ ]:
#Probability of any individual receiving the new page
df2.query('landing_page == "new_page"').count()[0]/df2.count()[0]

e. Use the results in the previous two portions of this question to suggest if there is evidence that one page leads to more conversions?  

The probability of converting is almost the same for the two pages. Out of a population of 290584 persons, the two conversion groups are just 450 persons apart. This is just 0.2% in difference which is not a significant difference. From the investigated dataset there is no clear evidence that one page leads to more conversions.
Of course boostrapping could be taken to use, but this will be done in the next section.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, I could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is, should I stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long should I run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now the decision will be based on all the data available.  Assuming that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what is the null and alternative hypotheses then?  

$H_0 = p_{new} \leq p_{old}$   
$H_1 = p_{new} > p_{old}$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

A  sample size equal to the ones in **ab_data.csv** will be used for each page.  <br><br>

A sampling distribution will be performed for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

a. What is the **convert rate** for $p_{new}$ under the null?   
b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [ ]:
#The conversion rate for the new
p_new_real = df2.query('landing_page == "new_page"')['converted'].mean()
p_new = df2.query('converted == 1').count()[0]/df2.count()[0]
p_new

In [ ]:
#The conversion rate for the old
p_old_real = df2.query('landing_page == "old_page"')['converted'].mean()
p_old = df2.query('converted == 1').count()[0]/df2.count()[0]
p_old

In [ ]:
#Computing the difference between the real observed conversion rates
p_diff_real = p_new_real-p_old_real
p_diff_real

c. What is $n_{new}$?

In [ ]:
#The amount of users for the treatment group
n_new = df2.query('group == "treatment"').count()[0]
n_new

d. What is $n_{old}$?

In [ ]:
#The amount of users for the control group
n_old = df2.query('group == "control"').count()[0]
n_old

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
#Simulating the amount of conversions given the amount of users and the possibility of conversion
new_page_converted = np.random.choice([0,1], size=n_new, replace=True, p=[1-p_new,p_new])
np.bincount(new_page_converted)

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
#Simulating the amount of conversions given the amount of users and the possibility of conversion
old_page_converted = np.random.choice([0,1], n_old, replace=True, p=[1-p_old,p_old])
np.bincount(old_page_converted)

g. Find $p_{new}$ - $p_{old}$ for the simulated values from part (e) and (f).

In [ ]:
#The difference in the simulated values is found
obs_diff = np.mean(new_page_converted)-np.mean(old_page_converted) 
obs_diff

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to parts **a. through g.**. All 10,000 values will be stored in **p_diffs**.

In [ ]:
#10,000 simulations are run
p_diffs = []
for _ in range(10000):
    new_p2 = np.random.choice([0,1], size=n_new,replace=True, p=[1-p_new,p_new])
    old_p2 = np.random.choice([0,1], n_old, replace=True, p=[1-p_old,p_old])
    p_diffs.append(np.mean(new_p2) - np.mean(old_p2))
p_diffs = np.array(p_diffs)


i. A histogram of the **p_diffs** will be plotted.

In [ ]:
plt.hist(p_diffs);
plt.axvline(x=p_diff_real, color='red'); 

The red line is the difference between the $p_{new}$ - $p_{old}$ values, which is very close to zero.   
The blue distribution is the 10,000 simulated cases, showing the expected standard distribution when considering the Central Limit Theorem. 

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
#Finding the mean of the simulated datapoints that are higher than the observed difference in ab_data.csv 
(p_diffs > p_diff_real).mean()


k. What was just computed in part **j.**?.  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

This is the p-value. The P-value shows that 90% of the simulated conversions are above the real difference observed. As the p-value is much larger than the used Type I error rate $\alpha$ = 0.05, then we fail to reject the $H_0$ hypothesis.

l. There exists a built-in function to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Below, the number of conversions for each page is calculated, as well as the number of individuals who received each page. Here `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm
#Finding the number of conversions for each page 
convert_old = df2.query('group == "control" and converted == 1').count()[0]
convert_new = df2.query('group == "treatment" and converted == 1').count()[0]
convert_old, convert_new, n_old, n_new  

m. Now using the `stats.proportions_ztest` to compute the test statistic and p-value. 

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new],alternative = 'smaller')
#The z-score, p-value and the significance of the z-score is computed.  
z_score, p_value, norm.cdf(z_score)

In [ ]:
#The critical level for a Type I error of 5% for a one-sided z-test
norm.ppf(1-(0.05))

n. What do the z-score and p-value computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

The significance of the z-score is 0.91, which is below the critical level for a one-sided test at 1.65 for a Type I error $\alpha$ = 5%. Again, we fail to reject the $H_0$ hypothesis, which agrees with the findings in part j and k.  
The P-value is the same as was found in part j.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, it is shown that the result achieved in the previous A/B test can also be achieved by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should be performed?

Since the dataset is binary, a logistic regression will be used

b. The goal is to use **statsmodels** to fit the regression model specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, first create a column for the intercept, and create a dummy variable column for which page each user received.  A **intercept** column, as well as an **ab_page** column will be added, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept'] = 1
# Column which is 1 when an individual receives the treatment and 0 if control
df2[['control','ab_page']] = pd.get_dummies(df2['group'])
df2.head()

c. Use **statsmodels** to import the regression model.  Instantiate the model, and fit the model using the two columns created in part **b.** to predict whether or not an individual converts.

In [ ]:
#The logit model is fitted
lm = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])  
results = lm.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in the **Part II**?<br><br>

The investigated hypotheses are:  
$H_0 = p_{new} - p_{old} \leq 0$  
$H_1 = p_{new} - p_{old} > 0$  
The P-value is 0.190 and since it is greater than the Type I error $\alpha$ = 0.05 then we again fail to reject the null hypothesis. Again, it is recommended to keep the old page.  
In Part II I tested using a one-sided z-test and in this regression a two-sided test has been used hence explaining the difference in P-values

f. Now, considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into the regression model.  Are there any disadvantages to adding additional terms into the regression model?

**Answer**  
Other factors than provided in this dataset could influence this A/B test. The actual interesting parameters will always depend on the investigated case.
When adding other parameters to the model you should be aware of potentially introducing multicollinearity, as the parameters could be depending on each other and then influence the results.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. 
Does it appear that country had an impact on conversion? 

In [ ]:
#The country dataset is read and joined with the existing dataset
df3 = pd.read_csv('countries.csv')
df3.head()

In [ ]:
# Maerge two files
df4 = df3.set_index('user_id').join(df2.set_index('user_id'))
df4.head()

In [ ]:
#Different countries in the dataset
df4['country'].value_counts()

In [ ]:
#Getting dummies
df4[['CA','UK','US']] = pd.get_dummies(df4['country'])
df4.head()

In [ ]:
#The logit model is fitted to the data
lm = sm.Logit(df4['converted'],df4[['intercept','UK','US']])
results = lm.fit()
results.summary()

The countries have a P-value above the Type I error rate, which indicates that they do not have a significant influence on the conversion. We still fail to reject the null hypothesis.

h. Though after looking at the individual factors of country and page on conversion, now it is time to look at an interaction between page and country to see if there are significant effects on conversion. 

In [ ]:
df4[['new','old']] = pd.get_dummies(df4['landing_page'])
df4.head()

In [ ]:
lm = sm.Logit(df4['converted'],df4[['intercept','new','UK','US']])
results = lm.fit()
results.summary()

No significant effects are seen on the conversion and we still fail to reject the null hypothesis.